# UX for LLMPop - Working Simultanously with Multiple LLMs, Locally, Safely, for Free
<a target="_blank" href="https://colab.research.google.com/github/LiorGazit/llmpop/blob/main/notebooks/multi_llm_webapp.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a> (pick a GPU Colab session for fastest computing)  

This notebook is a minimal “click, pick, and prompt” UI for LLMPop that lets you select up to four models and compare their replies side by side.  
It runs entirely free in your free Google Colab session and auto-handles local models via Ollama (no local installs on your machine).  
Use the optional OpenAI API key only if you choose the gpt-4o tab, otherwise everything stays within your Colab runtime. 
It’s meant for quick demos and teaching, not production, so you can show friends how to use LLMs without touching code.

### Setting up:

In [ ]:
# =====================  ONE-CELL, SCHEMA-SAFE APP (HTML chats, string stores)  =====================
import sys, subprocess
subprocess.run(
    [sys.executable, "-m", "pip", "install", "--quiet", "--upgrade", "gradio==4.44.1", "llmpop"],
    stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True
)

import os, time, json, html
os.environ["GRADIO_ANALYTICS_ENABLED"] = "false"  # quieter logs

import gradio as gr
from llmpop import init_llm  # unified factory

print("Gradio version:", gr.__version__)

# -------------------- Constants & mappings --------------------
TAB_COLORS = ["#000000", "#0A3D91", "#006400", "#FF1493"]  # black, dark blue, dark green, pink

CHOICE_LABELS = [
    "llama3.2:1b — Free & local (XXS)",
    "codellama — Free & local (S)",
    "deepseek-r1 — Free & local (S)",
    "gpt-4o — OpenAI (API key)",
]
LABEL_TO_ID = {
    CHOICE_LABELS[0]: "llama3.2:1b",
    CHOICE_LABELS[1]: "codellama",
    CHOICE_LABELS[2]: "deepseek-r1",
    CHOICE_LABELS[3]: "gpt-4o",
}
DEFAULT_LABELS = CHOICE_LABELS[:]

def make_status(msg, color=None):
    clr = color or "#444"
    return f'<div style="font-family:system-ui,sans-serif;font-size:14px;color:{clr};">{msg}</div>'

def render_chat(history, slot_idx):
    """history = list of {'role','content'} dicts -> colored HTML."""
    color_user = "#333"
    color_assistant = TAB_COLORS[slot_idx]
    rows = []
    for m in history:
        role = m.get("role", "")
        content = html.escape(m.get("content", ""))
        if role == "user":
            rows.append(f'<div style="margin:6px 0;padding:8px 10px;border-radius:10px;background:#f1f5f9;color:{color_user};"><b>You:</b> {content}</div>')
        else:
            rows.append(f'<div style="margin:6px 0;padding:8px 10px;border-radius:10px;background:#fff0; border:1px solid {color_assistant};color:{color_assistant};"><b>Model:</b> {content}</div>')
    return "<div>" + "".join(rows) + "</div>"

# -------------------- Model cache & helpers --------------------
_model_cache = {}

def get_model(model_name, provider, api_key, status_acc, color):
    key = (model_name, provider, api_key or "")
    if key in _model_cache:
        return _model_cache[key]

    if provider == "openai":
        if not api_key:
            status_acc.append(make_status(f"OpenAI model '{model_name}' selected but no API key provided.", "#B00020"))
            return None
        status_acc.append(make_status(f"Initializing OpenAI model: {model_name}…", color))
        mdl = init_llm(model=model_name, provider="openai",
                       provider_kwargs={"api_key": api_key}, temperature=0.0, verbose=False)
        status_acc.append(make_status(f"Ready: {model_name}", color))
    else:
        status_acc.append(make_status(
            f"Starting/connecting to Ollama and pulling '{model_name}' (first time takes a bit)…", color
        ))
        mdl = init_llm(model=model_name, provider="ollama",
                       provider_kwargs={"pull": True, "auto_install": True, "auto_serve": True},
                       temperature=0.0, verbose=False)
        status_acc.append(make_status(f"Ready: {model_name}", color))

    _model_cache[key] = mdl
    return mdl

def append_turn(history, user, assistant):
    history = history or []
    history += [{"role": "user", "content": user},
                {"role": "assistant", "content": assistant}]
    return history

# -------------------- UI --------------------
with gr.Blocks(
    title="LLMPop — Click · Pick · Prompt (Colab)",
    theme=gr.themes.Default(),
    css=f"""
    .llm-tab-0 .tabitem .tabs-label {{ color: {TAB_COLORS[0]} !important; }}
    .llm-tab-1 .tabitem .tabs-label {{ color: {TAB_COLORS[1]} !important; }}
    .llm-tab-2 .tabitem .tabs-label {{ color: {TAB_COLORS[2]} !important; }}
    .llm-tab-3 .tabitem .tabs-label {{ color: {TAB_COLORS[3]} !important; }}
    .slot-header-0 {{ color:{TAB_COLORS[0]}; font-weight:700; }}
    .slot-header-1 {{ color:{TAB_COLORS[1]}; font-weight:700; }}
    .slot-header-2 {{ color:{TAB_COLORS[2]}; font-weight:700; }}
    .slot-header-3 {{ color:{TAB_COLORS[3]}; font-weight:700; }}
    """
) as demo:
    gr.Markdown("# Experiment with local LLMs in your private environment\nPick LLMs on the left, tick which to invoke, then prompt below.")

    with gr.Row():
        # ---- Left panel ----
        with gr.Column(scale=1, min_width=320):
            gr.Markdown("### LLM picks")
            model_dd, model_ck = [], []

            for i, default_label in enumerate(DEFAULT_LABELS):
                with gr.Row():
                    dd = gr.Dropdown(
                        choices=CHOICE_LABELS,
                        value=default_label,
                        label=f"Slot {i+1} model",
                        allow_custom_value=False,
                        filterable=False,
                    )
                    ck = gr.Checkbox(value=False, label="Invoke on next prompt")
                model_dd.append(dd)
                model_ck.append(ck)

            api_key = gr.Textbox(
                label="OpenAI API Key (optional, for GPT-4o)",
                placeholder="sk-…",
                type="password",
                lines=1,
            )

            gr.Markdown("### Status")
            status_box = gr.HTML(make_status("Idle.", "#666"))

        # ---- Right panel ----
        with gr.Column(scale=2, min_width=680):
            tabs = gr.Tabs()
            slot_headers = []
            conv_html = []      # HTML render of chat per slot
            hist_store = []     # hidden string stores (JSON list of messages)

            for i in range(4):
                with gr.Tab(f"LLM Slot {i+1}", elem_classes=[f"llm-tab-{i}"]):
                    header = gr.HTML(f'<div class="slot-header-{i}">Model: <b>{DEFAULT_LABELS[i]}</b></div>')
                    html_box = gr.HTML("")   # output-only chat render
                    hidden_store = gr.Textbox(value="[]", visible=False)  # stringified JSON

                    slot_headers.append(header)
                    conv_html.append(html_box)
                    hist_store.append(hidden_store)

            prompt = gr.Textbox(label="Prompt", placeholder="Type your prompt here…", lines=6)
            send_btn = gr.Button("Send")

    # Header updates & tab reset (clear both chat render + store)
    def header_html(label, idx):
        return f'<div class="slot-header-{idx}">Model: <b>{label}</b></div>'

    for i, dd in enumerate(model_dd):
        dd.change(fn=lambda label, idx=i: header_html(label, idx), inputs=dd, outputs=slot_headers[i])
        dd.change(fn=lambda: "", inputs=None, outputs=conv_html[i])
        dd.change(fn=lambda: "[]", inputs=None, outputs=hist_store[i])

    # -------------------- Send (generator; histories via hidden Textbox strings) --------------------
    def send(
        p_text,
        h1, h2, h3, h4,            # stringified JSON histories
        l1, l2, l3, l4,            # dropdown labels
        c1, c2, c3, c4,            # checkboxes
        openai_key
    ):
        # parse stores -> lists
        def parse_hist(s):
            try:
                v = json.loads(s or "[]")
                return v if isinstance(v, list) else []
            except Exception:
                return []
        histories = [parse_hist(h1), parse_hist(h2), parse_hist(h3), parse_hist(h4)]
        labels    = [l1, l2, l3, l4]
        checks    = [c1, c2, c3, c4]

        def emit(status_html):
            # outputs: conv_html(4), hist_store(4), status
            return (
                render_chat(histories[0], 0),
                render_chat(histories[1], 1),
                render_chat(histories[2], 2),
                render_chat(histories[3], 3),
                json.dumps(histories[0]),
                json.dumps(histories[1]),
                json.dumps(histories[2]),
                json.dumps(histories[3]),
                status_html
            )

        if not any(checks):
            return emit(make_status("No LLM selected. Tick at least one checkbox to invoke.", "#B00020"))
        if not p_text or not p_text.strip():
            return emit(make_status("Please enter a prompt.", "#B00020"))

        statuses = []
        for idx in range(4):
            if not checks[idx]:
                continue

            label = labels[idx]
            model_id = LABEL_TO_ID.get(label)
            if not model_id:
                statuses.append(make_status(f"[Slot {idx+1}] Invalid model selection.", "#B00020"))
                continue

            provider = "openai" if model_id == "gpt-4o" else "ollama"
            color = TAB_COLORS[idx]

            if provider == "openai" and not (openai_key or "").strip():
                statuses.append(make_status(f"[Slot {idx+1}] '{model_id}' selected but no OpenAI API key provided.", "#B00020"))
                continue

            statuses.append(make_status(f"[Slot {idx+1}] Preparing {model_id}…", color))

            try:
                mdl = get_model(model_id, provider, (openai_key or "").strip() or None, statuses, color)
                if mdl is None:
                    continue

                statuses.append(make_status(f"[Slot {idx+1}] Invoking {model_id}…", color))

                t0 = time.time()
                out = mdl.invoke(p_text).content
                dt = time.time() - t0

                histories[idx] = append_turn(histories[idx], p_text, out)
                statuses.append(make_status(f"[Slot {idx+1}] {model_id} finished in {dt:.1f}s.", color))

            except Exception as e:
                statuses.append(make_status(f"[Slot {idx+1}] {model_id} error: {e!s}", "#B00020"))

        return emit("".join(statuses) if statuses else make_status("Done.", "#666"))

    # Button wiring: inputs = prompt + *string stores* + dropdowns + checkboxes + key
    send_btn.click(
        fn=send,
        inputs=[prompt, *hist_store, *model_dd, *model_ck, api_key],
        outputs=[*conv_html, *hist_store, status_box],
        concurrency_limit=1,
    )

# Queue + Launch
demo.queue(max_size=8)

Gradio version: 4.44.1


Gradio Blocks instance: 13 backend functions
--------------------------------------------
fn_index=0
 inputs:
 |-<gradio.components.dropdown.Dropdown object at 0x7ef489b35a30>
 outputs:
 |-<gradio.components.html.HTML object at 0x7ef489df9fa0>
fn_index=1
 inputs:
 outputs:
 |-<gradio.components.html.HTML object at 0x7ef48a00a9f0>
fn_index=2
 inputs:
 outputs:
 |-<gradio.components.textbox.Textbox object at 0x7ef489ba3a40>
fn_index=3
 inputs:
 |-<gradio.components.dropdown.Dropdown object at 0x7ef489bbd280>
 outputs:
 |-<gradio.components.html.HTML object at 0x7ef489ba3200>
fn_index=4
 inputs:
 outputs:
 |-<gradio.components.html.HTML object at 0x7ef489ba2f30>
fn_index=5
 inputs:
 outputs:
 |-<gradio.components.textbox.Textbox object at 0x7ef489ba2ff0>
fn_index=6
 inputs:
 |-<gradio.components.dropdown.Dropdown object at 0x7ef489ba3020>
 outputs:
 |-<gradio.components.html.HTML object at 0x7ef489a28260>
fn_index=7
 inputs:
 outputs:
 |-<gradio.components.html.HTML object at 0x7ef489a2a0

### Running the app:

In [ ]:
demo.launch(share=True, debug=True, max_threads=1)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://96e7fbc4b04d37da75.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


All done setting up Ollama (ChatOllama).

All done setting up Ollama (ChatOllama).

All done setting up Ollama (ChatOllama).

